In [3]:
#!pip install pymc3

In [28]:
import pandas as pd
import numpy as np
import pymc3 as pm
print(f"Running on PyMC3 v{pm.__version__}")

Running on PyMC3 v3.11.5


In [108]:
# True parameter values
alpha, sigma = 1, 1
beta = [1, 2.5]

# Size of dataset
size = 100

# Predictor variable
X1 = np.random.randn(size)
X2 = np.random.randn(size) * 0.2

# Simulate outcome variable
Y = alpha + beta[0] * X1 + beta[1] * X2 + np.random.randn(size) * sigma

In [4]:
basic_model = pm.Model()

with basic_model:

    # Priors for unknown model parameters
    alpha = pm.Normal("alpha", mu=0, sigma=10)
    beta = pm.Normal("beta", mu=0, sigma=10, shape=2)
    sigma = pm.HalfNormal("sigma", sigma=1)

    # Expected value of outcome
    mu = alpha + beta[0] * X1 + beta[1] * X2

    # Likelihood (sampling distribution) of observations
    Y_obs = pm.Normal("Y_obs", mu=mu, sigma=sigma, observed=Y)

In [8]:
map_estimate = pm.find_MAP(model=basic_model)
map_estimate

{'alpha': array(0.87558617),
 'beta': array([1.18966251, 3.57038781]),
 'sigma_log__': array(-0.08436807),
 'sigma': array(0.9190929)}

In [11]:
with basic_model:
    # draw 500 posterior samples
    trace = pm.sample(2000, return_inferencedata=False)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, beta, alpha]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 25 seconds.


In [27]:
trace["beta"]

array([[0.90127417, 1.27490666],
       [0.94235057, 2.86501573],
       [0.99859892, 2.39862491],
       ...,
       [0.7467961 , 2.32236062],
       [0.84382038, 2.82317467],
       [1.03732029, 1.66527842]])

In [15]:
np.quantile(trace["beta"],[0.025,0.05,0.5,0.95,0.975])
#Y = alpha + pm.math.dot(X, beta)

array([0.78374864, 0.81936746, 1.14192798, 2.84393346, 3.01718768])

In [218]:
# True parameter values
beta0, sigma = 1.1, 1
beta1 = -0.89
w = np.array([0.5, 0.20, 0.20, 0.05, 0.05])
delta = [-0.7, 0.3]

# Size of dataset
size = 236

In [219]:
# Predictor variable
X = np.random.randn(size,5)
bwqs = np.dot(X,w)

In [220]:
age = np.random.uniform(18,41, size)
gage = np.random.uniform(34,42, size)

# Simulate outcome variable
Y = beta0 + beta1 * bwqs + delta[0] * age + delta[1] * gage + np.random.randn(size) * sigma

In [223]:
bwqs_model = pm.Model()

with bwqs_model:

    # Priors for unknown model parameters
    beta0 = pm.Normal("beta0", mu=0, sigma=10)
    beta1 = pm.Normal("beta1", mu=0, sigma=10)
    delta = pm.Normal("delta", mu=0, sigma=10, shape=2)
    w_dir = pm.Dirichlet("w_dir", a=np.ones(5))
    sigma = pm.HalfNormal("sigma", sigma=1)

    # Expected value of outcome
    bwqs = pm.math.dot(X,w_dir)
    mu = beta0 + beta1 * bwqs + delta[0] * age + delta[1] * gage

    # Likelihood (sampling distribution) of observations
    Y_obs = pm.Normal("Y_obs", mu=mu, sigma=sigma, observed=Y)

In [224]:
with bwqs_model:
    # draw 2000 posterior samples
    trace = pm.sample(2000, return_inferencedata=False)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, w_dir, delta, beta1, beta0]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 84 seconds.
The acceptance probability does not match the target. It is 0.8789071336547054, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.8806714549908061, but should be close to 0.8. Try to increase the number of tuning steps.


In [225]:
trace["beta1"].mean()

-1.1125873194671472

In [226]:
np.quantile(trace["beta1"],[0.025,0.05,0.5,0.95,0.975])

array([-1.39186441, -1.3465384 , -1.11185135, -0.88218768, -0.83874031])